# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution
from nevergrad.optimization import optimizerlib


# Scipy

### Run for one argument

In [3]:
def cube(x):
    return x**3

In [4]:
dd = np.array([1,2,3])
cube(dd)

array([ 1,  8, 27])

In [5]:
differential_evolution(cube, [(-5, 5)], workers=1)

     fun: array([-125.])
     jac: array([74.99999981])
 message: 'Optimization terminated successfully.'
    nfev: 229
     nit: 14
 success: True
       x: array([-5.])

In [7]:
differential_evolution(cube, [(-5, 5)], workers=map, updating='deferred')

     fun: array([-125.])
     jac: array([74.99999981])
 message: 'Optimization terminated successfully.'
    nfev: 184
     nit: 11
 success: True
       x: array([-5.])

### Run With Map

In [43]:
from scipy.optimize._differentialevolution import (DifferentialEvolutionSolver)

In [44]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    rosen, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [48]:
solver.x

array([1., 1., 1., 1., 1.])

### Test With Chosen Function
We want to get an array and return a value for multiple parameters

In [58]:
bounds = [(1,2), (-10, 2), (0, 5)]

In [51]:
def cube_arr(x):
    return sum(x[:]**3)

In [68]:
cube_arr(np.array([1,2,3]))

36

In [62]:
with mp.Pool(10) as p, DifferentialEvolutionSolver(
    cube_arr, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [63]:
solver.x

array([  1., -10.,   0.])

### Try with two parameters

In [73]:
# with mp.Pool(10) as p, DifferentialEvolutionSolver(
#     cube_arr_two, bounds, updating='deferred', workers=p.map) as solver:
#     solver.solve()

### Test Args Tuple is Passed

In [74]:
# test that the args tuple is passed to the cost function properly.
bounds = [(-10, 10)]
args = (1., 2., 3.)

In [75]:
def quadratic(x, *args):
    if type(args) != tuple:
        raise ValueError('args should be a tuple')
    return args[0] + args[1] * x + args[2] * x**2.

In [76]:
result = differential_evolution(quadratic,
                                bounds,
                                args=args,
                                polish=True)
result.fun

array([0.66666667])

In [82]:
# test that the args tuple is passed to the cost function properly.
bounds = [(-7, 10)]
args = (1., 2., 3.)

def add_args(x, *args):
    return args[0] + x

In [83]:
result = differential_evolution(add_args,
                                bounds,
                                args=args,
                                polish=True)
result.fun

array([-6.])

### Two inputs with two boundsb

In [110]:
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))

    if x[0]+x[1] > 4.1: #this is the constraint, where you would say a+b+c <=1000
        return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e
    else:
        return 1000 #some high value

bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(ackley, bounds)
result.x, result.fun

(array([2.03967223, 2.0606556 ]), 6.862567561068813)

In [122]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [124]:
bounds = [(-5, 5), (0, 10)]
result = differential_evolution(test, bounds)
result.x, result.fun

(array([-5., 10.]), -15.0)

### Two Input Two Bounds and map

In [125]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [130]:
solver.x

array([-5., 10.])

In [133]:
test(solver.x)

-15.0